# UFoE Full 실험 — Google Colab

**Colab에서 바로 실행:** 아래 셀을 **위에서부터 순서대로** 실행하세요. `Runtime → Run all` 로 한 번에 실행할 수 있습니다.

- **환경 설정 셀** 실행 후, 터미널처럼 한 셀에서 `!python run_colab.py` 로 파이프라인만 실행할 수도 있습니다.
- 기본값: **mock** 모드 (실제 MD·ESMFold 없이 빠르게 완료)
- 실제 API/MD: 파이프라인 셀에서 `run_mode='quick'`, `esmfold_mode='api'` 로 변경 (수 분~수십 분 소요)

## 1. 환경 설정 (클론 + 의존성 설치)

In [ ]:
# 1) 저장소 클론 (이미 있으면 건너뜀)
import os
from pathlib import Path

REPO_NAME = "UFoE-Protein-generation_cursor"
if not os.path.isdir(REPO_NAME):
    !git clone https://github.com/plantv-blip/UFoE-Protein-generation_cursor.git

# 2) 프로젝트 디렉터리로 이동 (Colab 기본 = content/)
cwd = Path(os.getcwd())
if REPO_NAME in str(cwd):
    project_dir = cwd / "ufoe_protein_experiment" if (cwd / "ufoe_protein_experiment").exists() else cwd
else:
    project_dir = cwd / REPO_NAME / "ufoe_protein_experiment"
os.chdir(project_dir)
print("작업 디렉터리:", os.getcwd())

# 3) 의존성 설치 (requirements.txt 기준)
!pip install -q -r requirements.txt
!pip install -q openmm 2>/dev/null || echo "OpenMM 미설치 — Mock 모드로 진행"

# 4) 로깅이 셀에 보이도록 설정
import sys, logging
logging.basicConfig(level=logging.INFO, format="%(message)s", stream=sys.stdout)
for _name in ("src.pipeline", "src.folding", "src.generator"):
    logging.getLogger(_name).setLevel(logging.INFO)

# 5) 이후 셀에서 사용할 repo 루트 경로
REPO_ROOT = Path(os.getcwd()).parent
print("REPO_ROOT:", REPO_ROOT)

## 2. 후보 서열 생성 (선택)

In [ ]:
# mock: 빠름. api: ESMFold API 호출 (느릴 수 있음)
!python3 generate_candidates.py --n 10 --type B --output candidates/ --esmfold mock

## 3. Full 파이프라인 실행

In [ ]:
# Colab에서 바로 실행되도록 경로 자동 감지
import sys
from pathlib import Path

project_dir = Path(".").resolve()
repo_root = project_dir.parent
claude_dir = repo_root / "claude code" / "ufoe_protein_experiment"

if claude_dir.is_dir():
    os.chdir(claude_dir)
    sys.path.insert(0, str(claude_dir))
    from src.pipeline.pipeline import Phase2Pipeline, PipelineConfig
    config = PipelineConfig(
        output_dir="output/colab_run",
        run_mode="mock",
        esmfold_mode="mock",
    )
    pipeline = Phase2Pipeline(config)
    results = pipeline.run_full()
    pipeline.save_results(results)
    Phase2Pipeline.print_summary(results)
else:
    # claude code 없을 때: ufoe_protein_experiment 단순 파이프라인
    sys.path.insert(0, str(project_dir))
    from src.pipeline.pipeline import run_mock_experiment, Phase2Pipeline
    results = run_mock_experiment(n=10)
    print("접힘 성공률:", results.get("fold_success_rate"), "| K/M:", results.get("K_over_M"))

## 4. 결과 시각화

In [ ]:
import matplotlib.pyplot as plt

if hasattr(results, 'ufoe_2a_report') and results.ufoe_2a_report and results.control_random_report:
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    ax[0].bar(['UFoE 2a', 'Control'], [results.ufoe_2a_report.success_rate, results.control_random_report.success_rate])
    ax[0].set_ylabel('접힘 성공률')
    if results.elimination_ufoe_2a and results.elimination_control:
        ax[1].bar(['UFoE 2a', 'Control'], [results.elimination_ufoe_2a.self_consistency_ratio, results.elimination_control.self_consistency_ratio])
        ax[1].set_ylabel('K/M 자기일관성')
    plt.tight_layout()
    plt.show()
elif isinstance(results, dict):
    fig, ax = plt.subplots(1, 1, figsize=(5, 4))
    ax.bar(['UFoE'], [results.get('fold_success_rate', 0)])
    ax.set_ylabel('접힘 성공률')
    plt.show()
else:
    print("시각화할 results가 없습니다. 파이프라인 셀을 먼저 실행하세요.")